## Imports

In [31]:
import sys
sys.path.append("..")

In [32]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import when, col, count, upper
from datetime import date
from helpers import SnowflakeHelper
import json
import os

In [33]:
snowflake_helper = SnowflakeHelper()
snowflake_config = './../helpers/snowflake_config.json'
session = snowflake_helper.create_snowpark_session(snowflake_config)

[INFO] No schema passed, using default schema SAFEGUARDING_NYC_SCHEMA_BRONZE for the session
[SUCCESS] Config file loaded successfully!
[SUCCESS] Snowspark Session created successfully on schema SAFEGUARDING_NYC_SCHEMA_BRONZE!


## Extracting Data

In [34]:
use_of_force_combined = session.table('SAFEGUARDING_NYC_SCHEMA_GOLD.use_of_force_combined')
sqf = session.table('SAFEGUARDING_NYC_SCHEMA_SILVER.SQF')

In [35]:
use_of_force_combined.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FORCETYPE"        |"INCIDENT PCT"  |"TRI INCIDENT NUMBER"  |"PATROL BOROUGH"  |"OCCURRENCE DATE"  |"FORCE AGAINST MOS"  |"AGE"  |"SUBJECT GENDER"  |"SUBJECT RACE"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------
|Physical Force     |1               |2020020962067          |MANHATTAN         |2020-05-06         |Physical Force       |NULL   |MALE              |BLACK           |
|Physical Force     |1               |2020020962143          |MANHATTAN         |2020-10-11         |Physical Force       |NULL   |MALE              |BLACK           |
|Physical Force     |5               |2020024962080          |MANHATTAN         |2020-06-30         |Physical Force       |NULL   |MALE              |BLACK     

In [36]:
sqf.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [39]:
use_of_force_combined.count()

29723

In [40]:
sqf.count()

101989

## Joining Use Of Forces Tables

### On PRECINCT

In [37]:
use_of_force_and_sqf_on_precinct = use_of_force_combined.join(
    sqf, 
    use_of_force_combined['INCIDENT PCT'] == sqf['STOP_LOCATION_PRECINCT'], 
    how="inner"
)
use_of_force_and_sqf_on_precinct.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [38]:
use_of_force_and_sqf_on_precinct.count()

45736869

### On Borough

In [29]:
use_of_force_and_sqf_on_borough = use_of_force_combined.join(
    sqf, 
    use_of_force_combined['PATROL BOROUGH'] == sqf['STOP_LOCATION_PATROL_BORO_NAME'], 
    how="inner"
)
use_of_force_and_sqf_on_borough.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [30]:
use_of_force_and_sqf_on_borough.count()

725171410

## Exporting Data to Gold Medallion

In [23]:
table_name = "SAFEGUARDING_NYC_SCHEMA_GOLD.use_of_force_combined"
use_of_force_combined.write.saveAsTable(table_name, mode="overwrite")